In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import csv
from os.path import exists
from os import mkdir
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()),options=chrome_options)

In [ ]:
driver.get("https://airbnb.com.br/")
sleep(3)
try:
    anuncio = driver.find_element(By.XPATH,'//*[@aria-label="Fechar"]')
    anuncio.click()
except:
    pass

In [ ]:
caixa = driver.find_element(By.XPATH,'//div//*[contains(text(),"Qualquer lugar")]')
caixa.click()
sleep(2)

In [ ]:
busca = driver.find_element(By.CSS_SELECTOR,'.ivycze5')
busca.send_keys('Itaboraí',Keys.RETURN)
sleep(2)

# busca = driver.find_element(By.XPATH,'//*[@id="/homes-1-input"]')


In [ ]:
checkin = driver.find_element(By.XPATH,'//h2[contains(text(),"janeiro")]/ancestor::div[@class="_ytfarf"]//*[text()="20"]')
checkin.click()
sleep(2)
checkout = driver.find_element(By.XPATH,'//h2[contains(text(),"janeiro")]/ancestor::div[@class="_ytfarf"]//*[text()="23"]')
checkout.click()
sleep(2)
hospedes = driver.find_element(By.XPATH,'.//*[@id="search-tabpanel"]/div/div[5]/div[1]/div[1]/div/div[1]')
hospedes.click()
sleep(2)
adultos = driver.find_element(By.XPATH,'//*[@id="stepper-adults"]/button[2]')
adultos.click()
adultos.click()
sleep(2)
buscar = driver.find_element(By.CSS_SELECTOR,'._kaq6tx')
buscar.click()
sleep(5)

In [ ]:
qtdPag = driver.find_element(By.XPATH,'//*[@id="site-content"]/div[2]/div[3]/div/div/nav/div/a[4]')
qtdPag = int(qtdPag.text)

In [ ]:
if(not exists('data/')):
    os.makdir('data/')


with open('data/airbnb.csv', 'w', encoding="utf-8-sig",newline='') as airbnb_file:
    campos = [
        'nome', 
        'local',
        'precoNoite',
        'total',
        'taxas',
        'avaliacoes',
        'ondeDormir', 
        'url'
    ]

    csv_writer = csv.DictWriter(airbnb_file, fieldnames=campos, delimiter=';', quoting=csv.QUOTE_ALL)
    csv_writer.writeheader()
    
    tempo = 3

    for i in range(qtdPag):
        print("-------------------",i+1,'-------------------\b')
        acomodacoes = driver.find_elements(By.CSS_SELECTOR,'.lwy0wad')
        for acomodacao in acomodacoes:
            acomodacao.click()
            sleep(tempo)
            driver.switch_to.window(driver.window_handles[1])
            
            try:
                nome = driver.find_element(By.CSS_SELECTOR,'._1n81at5')
                nome = nome.text
            except:
                nome = "---"
                
            try:
                local = driver.find_element(By.CSS_SELECTOR,'._9xiloll')
                local = local.text
            except:
                local = "---"

            try:
#                 precoNoite = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]')
                precoNoite = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div/div/span/span').text.split()[0][2:]
                precoNoite = int(precoNoite)
            except:
                precoNoite = "---"
                
            try:
                total = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/section/div[2]/div/span[2]')
                total = int(total.text[2:])
            except:
                total = '---'
            
            if(total != "---" and precoNoite != "---"):
                taxas = total - precoNoite*3
            else:
                taxas = '---'
            
            try:
                avaliacoes = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span/span[2]')
                avaliacoes = avaliacoes.text
            except:
                avaliacoes = "---"

            try:
                ondeDormir = driver.find_elements(By.CSS_SELECTOR,'._1a5glfg')
                ondeDormir = "".join(list(map(lambda x: x.text+',',ondeDormir)))
                if(ondeDormir == ""):
                    ondeDormir = driver.find_elements(By.CSS_SELECTOR,'._4zdnhq')
                    ondeDormir = "".join(list(map(lambda x: x.text+',',ondeDormir)))

            except:
                ondeDormir = "---"

            url = driver.current_url

            print(nome,local,precoNoite, total, taxas, avaliacoes,ondeDormir,url)
            
            csv_writer.writerow(dict(zip(campos, [nome, local, precoNoite, total, taxas, avaliacoes, ondeDormir, url])))


            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
            
        if(i != (qtdPag-1)):
            proximo = driver.find_element(By.XPATH,'//a[@aria-label="Próximo"]')
            proximo.click()
            sleep(tempo)
        else:
            print("Fim!")